In [2]:
import pandas as pd

In [9]:
df = pd.read_feather("../data/open-powerlifting.feather")
df.head()

,Name,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,...,Tested,Country,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetTown,MeetName
0,Alona Vladi,F,SBD,Raw,33.0,24-34,24-39,O,58.30,60,...,Yes,Russia,None,GFP,None,2019-05-11,Russia,None,Bryansk,Open Tournament
1,Galina Solovyanova,F,SBD,Raw,43.0,40-44,40-49,M1,73.10,75,...,Yes,Russia,None,GFP,None,2019-05-11,Russia,None,Bryansk,Open Tournament
2,Daniil Voronin,M,SBD,Raw,15.5,16-17,14-18,T,67.40,75,...,Yes,Russia,None,GFP,None,2019-05-11,Russia,None,Bryansk,Open Tournament
3,Aleksey Krasov,M,SBD,Raw,35.0,35-39,24-39,O,66.65,75,...,Yes,Russia,None,GFP,None,2019-05-11,Russia,None,Bryansk,Open Tournament
4,Margarita Pleschenkova,M,SBD,Raw,26.5,24-34,24-39,O,72.45,75,...,Yes,Russia,None,GFP,None,2019-05-11,Russia,None,Bryansk,Open Tournament


In [10]:
df[df["MeetName"].str.contains("Battle of the Bay")].MeetName.unique()

array(["WARCAT's Battle of the Bay", 'Battle of the Bay',
       'Battle of the Bay VI', 'Battle of the Bay Powerlifting',
       'Battle of the Bay V', 'Battle of the Bay VIII',
       'Battle of the Bay Benchpress and Deadlift',
       'Drug Tested Battle of the Bay VIII', 'Battle of the Bay 3',
       'Battle of the Bay VII', 'Battle of the Bay II Powerlifting',
       'Battle of the Bay IV'], dtype=object)

In [11]:
df.isnull().sum()

Name                      0
Sex                       0
Event                     0
Equipment                 0
Age                  990410
AgeClass             750941
BirthYearClass       925670
Division               1454
BodyweightKg          34502
WeightClassKg         32025
Squat1Kg            1981742
Squat2Kg            1989435
Squat3Kg            2008724
Squat4Kg            2643542
Best3SquatKg         908685
Bench1Kg            1546782
Bench2Kg            1560292
Bench3Kg            1595849
Bench4Kg            2631997
Best3BenchKg         318690
Deadlift1Kg         1886269
Deadlift2Kg         1901177
Deadlift3Kg         1935438
Deadlift4Kg         2631995
Best3DeadliftKg      763744
TotalKg              183468
Place                     0
Dots                 206678
Wilks                206678
Glossbrenner         206678
Goodlift             422782
Tested               695589
Country             1178223
State               2168284
Federation                0
ParentFederation    